# Iron-Python Notebook for Processing Raw Hardware Results

This will be an all-in-one package for processing data/results of workload execution archives in specific directories in this repository.
It is assumed that all the results were captured using WorkloadExec-v2.py file, so that everything is archived appropriately.


## Imports

In [1]:

# General Packages
import os
import sys
import glob
import io
import shutil
## For handling BZ archive of result files
import requests
import tempfile
import tarfile

# For data processing 
import datetime
import re
import csv
import pandas


## Utility Functions

In [2]:
def extract_data(data_archive:str, tmp_working_dir: str) ->str:
    # TempDirectory to string returns something like ' <TemporaryDirectory ' prefixed
    ## for handling tmp directory cleanly, striping off these characters
    tmpfilePath = str(tmp_working_dir).split(' ')[1].replace('\'','').replace('>','')
    # print('Temporary directory used: '+tmpfilePath+'/')

    # Extract Files
    archive = tarfile.open(data_archive, 'r:bz2')
    # An assumption is made here that the archives are stored at directory level, than directly files
    # Hence getting the first level directory name 
    # e.g.: <TarInfo '11-03-2023_14-58-03_BigCore-10itr-50msSmplg-CPUFreq-2.0GHz' at 0x7fbc676ecd00>
    dirname=str(archive.getmembers()[0]).split(' ')[1].replace('\'','')

    archive.extractall(tmpfilePath+'/')
    archive.close()
    return os.path.join(tmpfilePath,dirname)

## Function to process Perf-stat output

In [3]:
# Define regular expressions to match the lines with data
# Regular expression for [1] & [2], and associated result variable(s)
re_1_2_firstlevel = re.compile(r"""
                                ^    # Line start
                                [#]+ # Immediate occurance of '#'
                                .*   # Anything following that
                            """, re.X)
re_1_timestamp_info = re.compile(r"""
                                ^[#]+                 # Line Start with # character
                                \s+started\s+on\s+    # This string will be present before timestamp
                                (.*)                  # Time of perf start
                                """, re.X)


# Regular expression for [4]
re_4_summary_header = re.compile(r"""^\s+
                                Performance\scounter\sstats\sfor\s
                                \'(.*)\'\s+              # Type of monitoring in perf
                                \((\d+)\sruns\):
                                """,re.X)

# Regular expression for [3]
re_3_perf_stat_record = re.compile(r"""^\s+
                                (\d+\.\d+)\s+          # Timestamp
                                (\S+)\s+(\d+)\s+       # CPU-ID, CPUs
                                (.*)                   # counts, [unit], events, event-info
                                """, re.X)
re_3_subrec_countfield_nc = re.compile(r"""
                                    <not\scounted>\s+     # Is it a <not counted field> ?
                                    (.*)
                                    """, re.X)
re_3_subrec_countfield = re.compile(r"""
                                    (\d+[.\d]*)\s+        # counts
                                    (.*)              # units or event-name
                                    [#]+(.*)                  # Event extra info
                                    """, re.X)

In [4]:
def Process_ProfFile(proffile:str, outcsvfile:str) -> None:
    with io.open(proffile, 'rt') as  proffile_entry:
        
        stats_gatherer = {}
        perf_stat_starttime = ''
        perf_stat_starttime_dateobj = None
        perf_stat_currtime_dateobj = None
        perf_stat_columns = []
        perf_stat_runtype=''
        perf_stat_runcount=0


        records_ended = False
        record_counter = 0
        prev_timestamp =''
        rec_timestamp = rec_cpuid = rec_cpucnt = rec_counter = rec_eventname = ''
        csvout = open(outcsvfile, 'w') 
        csvwriter = None
        #Decoding the prof/perf-stat file data
        for line in proffile_entry.readlines():
            # print (str(len(line))+':'+line)

            ## Okay so, its a line starting with # character
            if re_1_2_firstlevel.match(line):
                # is it a time stamp info line??
                re_1_match = re_1_timestamp_info.match(line)
                if (re_1_match):
                    perf_stat_starttime = re_1_match.group(1)
                    perf_stat_starttime_dateobj = datetime.datetime.strptime(perf_stat_starttime,'%a %b %d %H:%M:%S %Y')
                    perf_stat_starttime_pddateobj = pandas.Timestamp(perf_stat_starttime_dateobj, unit='ns')
                    perf_stat_currtime_dateobj = datetime.datetime.strptime(perf_stat_starttime,'%a %b %d %H:%M:%S %Y')
                    perf_stat_currtime_pddateobj = pandas.Timestamp(perf_stat_currtime_dateobj)
                    # print('Start Time:'+str(perf_stat_currtime_pddateobj))
                else:
                    # Then it must be a column header info line
                    perf_stat_columns = line[1:].split()
            else:
                # So its lines other than meta info ones ([1] & [2])
                # just to skip the last few records as we only need the perf-stat samples, not the summary at the end
                if (records_ended == False):
                    re_4_match = re_4_summary_header.match (line)
                    re_3_match = re_3_perf_stat_record.match(line)
                
                    if (re_4_match):
                        perf_stat_runtype  = re_4_match.group(1)
                        perf_stat_runcount = int(re_4_match.group(2))
                        records_ended = True
                        # Okay, so we have reached till the summary section
                        # write the last record
                        record_counter += 1
                        csvwriter.writerow(stats_gatherer)
                        stats_gatherer.clear()
                        
                    elif (re_3_match):
                        rec_timestamp = re_3_match.group(1)
                        rec_cpuid     = re_3_match.group(2)
                        rec_cpucnt    = re_3_match.group(3)


                        if (prev_timestamp==''):
                            prev_timestamp = rec_timestamp
                            ns = perf_stat_starttime_pddateobj+pandas.Timedelta(seconds=float(rec_timestamp))
                            # print('PDTime:'+str(ns)+' subsec: '+rec_timestamp)
                            stats_gatherer['utctime'] = ns

                        if (prev_timestamp != rec_timestamp):
                            ## Okay, we are encountering a new record, so dump the old one
                            record_counter += 1
                            if (record_counter == 1):
                                csvwriter =  csv.DictWriter(csvout, stats_gatherer.keys())
                                csvwriter.writeheader()
                            csvwriter.writerow(stats_gatherer)
                            stats_gatherer.clear()
                            prev_timestamp = rec_timestamp

                            ns = perf_stat_starttime_pddateobj+pandas.Timedelta(seconds=float(rec_timestamp))
                            # print('PDTime:'+str(ns)+' subsec: '+rec_timestamp)
                            stats_gatherer['utctime'] = ns

                        re3__ncsubrec_match = re_3_subrec_countfield_nc.match(re_3_match.group(4))

                        if (re3__ncsubrec_match):
                            trec = re3__ncsubrec_match.group(1)
                            # print (rec_cpuid + ' NC - '+trec)
                            stats_gatherer[rec_cpuid+'_'+trec] = 'NaN'
                            pass
                        else:

                            re_3_subrec_match = re_3_subrec_countfield.match(re_3_match.group(4))
                            if (re_3_subrec_match):
                                split_str =re_3_subrec_match.group(2).split() 
                                # print('event: '+re_3_subrec_match.group(1)+'-'+ split_str[len(split_str)-1] )
                                rec_counter = re_3_subrec_match.group(1)
                                rec_eventname = split_str[len(split_str)-1]
                            else:
                                ## Special cases, some instructions stats count seems to be having no '#' separator
                                ## Handle  it separately. e.g:
                                ##     808323      instructions                                                         (3.64%)
                                ##     52451588      instructions                                                         (8.98%)
                                ##     424251      instructions                                                         (8.92%)
                                ##     394060      instructions                                                         (8.86%)
                                tstr = re_3_match.group(4)
                                rec_counter   = tstr[0]
                                rec_eventname = tstr[1]
                                assert len(tstr) != 3,'Some invalid records structure found, please check!!!'
                                # print(re_3_match.group(4))

                            if (rec_eventname == 'cpu-clock'):
                                stats_gatherer[rec_cpuid+'_'+rec_eventname] = float(rec_counter)
                            else:
                                stats_gatherer[rec_cpuid+'_'+rec_eventname] = int(rec_counter)
                                
                else:
                    ## For now, we are only going to process the individual records, not the summary
                    ## So we can very well break out of the loop
                    break


        # print ('Start time: '+ perf_stat_starttime)
        # print ('Columns: '+ str(perf_stat_columns))
        # print ('runs type: '+ perf_stat_runtype)
        # print ('runs count: '+ str(perf_stat_runcount) )
        # print ('Total Records: '+ str(record_counter) )



# Generating Combined dataset per test results

In [5]:
def ProcessTestResults( data_perfstat_file:str,
                        data_power_file:str,
                        data_poll_file:str,
                        outputcsvfile:str,      # Absolute/Full path to output CSV file
                        tolerance_perf_to_power = pandas.Timedelta('50 milliseconds'),   # Tolerance value in UTC-time to merge perf-stat and SmartPower3 data
                        tolerance_perf_to_poll  = pandas.Timedelta('5 seconds'),
                       ) -> None:
    # Forming file names
    # data_prof = os.path.join(resultsdir,testnameprefix+'.prof')
    # data_power    = os.path.join(resultsdir,testnameprefix+'.powdata')
    # data_polldata = os.path.join(resultsdir,testnameprefix+'.polldata')
    temp_data_prof_csv  = data_perfstat_file+'csv'

    # Process the perf-stat file to generate intermediate CSV file for further processing
    Process_ProfFile(data_perfstat_file, temp_data_prof_csv)

    # Now read all the CSV files
    prof_df = pandas.read_csv(temp_data_prof_csv)
    power_df = pandas.read_csv(data_power_file)
    poll_df = pandas.read_csv(data_poll_file)

    # Drop unncessary columsn which are not required and not influencing current test scenarios
    power_df.drop([
                    'sm_mstime', 'localtime',
                    'ps_ippwr-volts_mV', 'ps_ippwr-ampere_mA', 'ps_ippwr-watt_mW', 'ps_ippwr-status_b',
                    'dev_ippwr-ch0-volts_mV', 'dev_ippwr-ch0-ampere_mA', 'dev_ippwr-ch0-watt_mW', 'dev_ippwr-ch0-status_b', 'dev_ippwr-ch0-interrupts',
                    'dev_ippwr-ch1-status_b', 'dev_ippwr-ch1-interrupts',
                    'crc8-2sc','crc8-xor'
                    ],
                axis=1, 
                inplace=True)
    poll_df.drop( [
                    'ts_local', 
                    'stat_cpuid', 'stat_user', 'stat_nice', 'stat_system', 'stat_idle', 'stat_iowait',
                    'stat_irq', 'stat_softirq', 'stat_steal', 'stat_guest', 'stat_guest_nice'
                ],
                axis=1, 
                inplace=True)

    # print ('Perf/Profile Data:')
    # display(prof_df)

    # print ('Power data:')
    # display(power_df)

    # print('Polled data')
    # display(power_df)

    prof_df["utctime"] = pandas.to_datetime(prof_df['utctime'])
    prof_df.index = prof_df["utctime"]

    power_df["utctime"] = pandas.to_datetime(power_df['utctime'])
    power_df.index = power_df["utctime"]

    poll_df["ts_utc"] = pandas.to_datetime(poll_df['ts_utc'])
    poll_df.index = poll_df["ts_utc"]

    
    df = pandas.merge_asof(left=prof_df, right=power_df,right_index=True,left_index=True,direction='nearest',tolerance=tolerance_perf_to_power)
    df = pandas.merge_asof(left=df, right=poll_df,right_index=True,left_index=True,direction='nearest',tolerance=tolerance_perf_to_poll)

    pandas.set_option('display.max_columns', None)
    pandas.set_option('display.expand_frame_repr', False)

    # print('Merged data')
    # display(df)

    print('Saving Merged data to CSV location: '+ outputcsvfile)
    df.to_csv(outputcsvfile,header=True)


## Main body to Iterate and process each results set

In [6]:
# direcotories to process , so just converting them to absolute paths
__results_archive_dirs__ = [
        '02-Idling-PerfSleep/MaxFan',
        '02-Idling-PerfSleep/NoFan',
        '03-Workloads'
    ]

__output_subdir__ = 'combined_dataset'
__output_dirs__ = []

results_archive_dirs = []
for dir in __results_archive_dirs__:
    results_archive_dirs.append(os.path.abspath('results/'+dir))
    __output_dirs__.append(os.path.abspath(__output_subdir__+'/'+dir))

# Now iteratively process the archives and create combined data set for each
total_dirs = len(results_archive_dirs)
curr_dir = 1
for curr_archive_dir in results_archive_dirs:
    print ('Processing ('+str(curr_dir)+'/'+str(total_dirs)+'): '+ curr_archive_dir)
    output_dir = __output_dirs__[curr_dir-1]
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    

    __res_archives = glob.glob(curr_archive_dir+'/*.tar.bz2')
    for archives in __res_archives:
        print ('Archive: '+ archives)
        output_subdir = os.path.splitext(os.path.splitext(os.path.basename(archives))[0])[0]
        output_subdir = os.path.join(output_dir,output_subdir)
        print ('Output to be placed in : '+ output_subdir)
        if not os.path.exists(output_subdir):
            os.makedirs(output_subdir)


        #Temporary files/directories for handling data
        tmpdir = tempfile.TemporaryDirectory()
        # print(tmpdir)
        
        data_dir = extract_data(archives,tmpdir)
        # print ('Temporary extraction to : '+data_dir)


        __raw_results = glob.glob(data_dir+'/*.prof')
        filecount = len(__raw_results)
        tctr = 1
        print('Number of results to process: '+str(filecount))
        for raw_result in __raw_results:
            __prefix_ = os.path.splitext(raw_result)[0]
            __basename__ = os.path.basename(raw_result)
            __outfile__ = os.path.join(output_subdir,__basename__+'.csv')

            pow_data_file = __prefix_+'.powdata'
            poll_data_file = __prefix_+'.polldata'
            try:
                print ('Processing  File ('+str(tctr)+'/'+str(filecount)+'): '+ __basename__)
                # print (raw_result, pow_data_file, tctr)
                ProcessTestResults(data_perfstat_file=raw_result,
                               data_power_file=pow_data_file,
                               data_poll_file=poll_data_file,
                               outputcsvfile=__outfile__
                               )
            except ValueError as e:
                print ('Invalid data entry found in '+str(raw_result))
            tctr += 1
    curr_dir += 1


Processing (1/3): /home/vaisakh/developer/modeling/E0240_ModSim-PowerMon/results/02-Idling-PerfSleep/MaxFan
Archive: /home/vaisakh/developer/modeling/E0240_ModSim-PowerMon/results/02-Idling-PerfSleep/MaxFan/11-14-2023_07-21-05_IdlePerfSleepWorkload-MaxFan-LittleCore-20sidle-CPUFreq-0.4GHz.tar.bz2
Output to be placed in : /home/vaisakh/developer/modeling/E0240_ModSim-PowerMon/combined_dataset/02-Idling-PerfSleep/MaxFan/11-14-2023_07-21-05_IdlePerfSleepWorkload-MaxFan-LittleCore-20sidle-CPUFreq-0.4GHz
Number of results to process: 2
Processing  File (1/2): IdleSleep-perf-1.prof
Saving Merged data to CSV location: /home/vaisakh/developer/modeling/E0240_ModSim-PowerMon/combined_dataset/02-Idling-PerfSleep/MaxFan/11-14-2023_07-21-05_IdlePerfSleepWorkload-MaxFan-LittleCore-20sidle-CPUFreq-0.4GHz/IdleSleep-perf-1.prof.csv
Processing  File (2/2): IdleSleep-perf-2.prof
Saving Merged data to CSV location: /home/vaisakh/developer/modeling/E0240_ModSim-PowerMon/combined_dataset/02-Idling-PerfSleep

In [ ]:
# Compress the combined_dataset
source_dir = 'combined_dataset'
with tarfile.open('combined_dataset.tar.bz2', "w:bz2") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
#shutil.rmtree(os.path.abspath(__output_subdir__))